In [1]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from utils import get_smiles_encodings, load_data, smile_to_hot
from model_regression import LinearRegression_RidgeRegression
from sklearn.linear_model import SGDRegressor
from datetime import datetime
import numpy as np
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
'''https://www.youtube.com/watch?v=0ARLObJJ3y4&t=661s'''

In [2]:
file_smiles = './dataset/QM9.txt'
file_properties = './dataset/properties_QM9.npz'
smiles, alphabet, largest_molecule_len = get_smiles_encodings(file_smiles)
properties = np.load(file_properties)['properties'].astype(np.float32)

X_train, X_test, y_train, y_test = load_data(smiles, properties)
print("X - train : ", X_train.shape)
print("y - train : ", y_train.shape)
X_hot = np.array([smile_to_hot(x, largest_molecule_len, alphabet)[1].sum(axis=0) for x in X_train])
#print("X_hot", X_hot)
X_hot_test = np.array([smile_to_hot(x, largest_molecule_len, alphabet)[1].sum(axis=0) for x in X_test])
#print("X_hot_test", X_hot_test)
print("X_hot : ", X_hot.shape)
print("X_hot_test : ", X_hot_test.shape)
print("y_train", y_train)
print("y_test",y_test)



X - train :  (92428,)
y - train :  (92428, 4)
X_hot :  (92428, 14)
X_hot_test :  (39612, 14)
y_train [[  0.6361    0.       16.043     0.     ]
 [  0.162     0.       17.031     0.     ]
 [ -0.8247    0.       18.015     0.     ]
 ...
 [ -0.5388    0.      130.143     1.     ]
 [ -1.7719    0.      131.131     1.     ]
 [ -0.89423   0.      131.131     1.     ]]
y_test [[ -1.3449   0.     132.115    1.    ]
 [  1.9317   0.     124.183    1.    ]
 [  1.0462   0.     125.171    1.    ]
 ...
 [ -0.8808   0.     120.155    8.    ]
 [  0.3187   0.     119.167    8.    ]
 [ -0.55     0.     121.139    8.    ]]


In [5]:
y_test_0 = y_test[:,0]
y_test_1 = y_test[:,1]
y_test_2 = y_test[:,2]
y_test_3 = y_test[:,3]  

In [7]:
loss_function = ["ls", "lad", "huber", "quantile"]
learning_rate_ = [10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
best_0 = [100,0,0]
best_1 = [100,0,0]
best_2 = [100,0,0]
best_3 = [100,0,0]

#we iterate over loss function and learning rates
for loss in loss_function:
    for lr in learning_rate_:
        # 0 
        reg = GradientBoostingRegressor(random_state=0, learning_rate=lr, loss=loss)
        reg.fit(X_hot,  y_train[:,0])
        y_pred_0 = reg.predict(X_hot_test)
        
        # 1
        reg = GradientBoostingRegressor(random_state=0, learning_rate=lr, loss=loss)
        reg.fit(X_hot,  y_train[:,1])
        y_pred_1 = reg.predict(X_hot_test)
        y_pred_1 = np.round(y_pred_1)

        # 2 
        reg = GradientBoostingRegressor(random_state=0, learning_rate=lr, loss=loss)
        reg.fit(X_hot,  y_train[:,2])
        y_pred_2 = reg.predict(X_hot_test)
        
        # 3 
        reg = GradientBoostingRegressor(random_state=0, learning_rate=lr, loss=loss)
        reg.fit(X_hot,  y_train[:,3])
        y_pred_3 = reg.predict(X_hot_test)
        y_pred_3 = np.round(y_pred_3)
        
        # calcul res
        sum_0 = 0
        sum_1 = 0
        sum_2 = 0
        sum_3 = 0
        for i in range(y_test_0.shape[0]):

            # 0
            sum_0 = sum_0 + np.abs(np.abs(y_pred_0[i]) - np.abs(y_test_0[i]))

            # 1
            if(y_pred_1[i] == y_test_1[i]):
                sum_1 = sum_1 + 1

            # 2
            sum_2 = sum_2 + np.abs(np.abs(y_pred_2[i]) - np.abs(y_test_2[i]))

            # 3
            if(y_pred_3[i] == y_test_3[i]):
                sum_3 = sum_3 + 1

        sum_0 = sum_0 / y_test_0.shape[0]
        sum_1_r = 100 - (sum_1 / y_test_1.shape[0] * 100)
        sum_2 = sum_2 / y_test_2.shape[0]
        sum_3_r = 100 - (sum_3 / y_test_3.shape[0] * 100)
        
        
        # change res if needed
        if sum_0 < best_0[0]:
            best_0 = [sum_0, loss, lr]

        if sum_1_r < best_1[0]:
            best_1 = [sum_1_r, loss, lr]
            
        if sum_2 < best_2[0]:
            best_2 = [sum_2, loss, lr]

        if sum_3_r < best_3[0]:
            best_3 = [sum_3_r, loss, lr]

        print(loss, " ", lr, "   - done")
        
print("OPTIMAL RESULTS")                
print(best_0)
print(best_1)
print(best_2)
print(best_3)
        

ls   10    - done
ls   1    - done
ls   0.1    - done
ls   0.01    - done
ls   0.001    - done
ls   0.0001    - done
ls   1e-05    - done
lad   10    - done
lad   1    - done
lad   0.1    - done
lad   0.01    - done
lad   0.001    - done
lad   0.0001    - done
lad   1e-05    - done
huber   10    - done
huber   1    - done
huber   0.1    - done
huber   0.01    - done
huber   0.001    - done
huber   0.0001    - done
huber   1e-05    - done
quantile   10    - done
quantile   1    - done
quantile   0.1    - done
quantile   0.01    - done
quantile   0.001    - done
quantile   0.0001    - done
quantile   1e-05    - done

[0.26763972449390216, 'lad', 1]
[58.43178834696557, 'ls', 1]
[0.6562325422345359, 'ls', 1]
[5.399878824598602, 'huber', 1]


In [12]:
properties_txt = ['logP: represents a measure of the tendency of a compound to move from the aqueous phase into lipids',
                  'Number of rotatable bonds (RBN): the number of bonds which allow free rotation around themselves',
                  'Molecular weight (MW): the weight of a molecule based on the atomic masses of all atoms in the molecule',
                  'Number of the rings (RN): the number of connected sets of atoms and bonds in which every atom and bond is a member of a cycle']

print("LOSS DEF : Loss function to be optimized. ‘ls’ refers to least squares regression. ‘lad’ (least absolute deviation) is a highly robust loss function solely based on order information of the input variables. ‘huber’ is a combination of the two. ‘quantile’ allows quantile regression (use alpha to specify the quantile).")
print()


print(properties_txt[0])
print()
print("model -> loss_function : ", best_0[1], " - learning rate : ", best_0[2])
print()
print("Error is on avearage : ", best_0[0])
print("Let's say the target value is : 1.0, then the model would guess a value in the following interval :")
print(" -> [", 1.0 - best_0[0],",",1.0 + best_0[0],"]")

LOSS DEF : Loss function to be optimized. ‘ls’ refers to least squares regression. ‘lad’ (least absolute deviation) is a highly robust loss function solely based on order information of the input variables. ‘huber’ is a combination of the two. ‘quantile’ allows quantile regression (use alpha to specify the quantile).

logP: represents a measure of the tendency of a compound to move from the aqueous phase into lipids

model -> loss_function :  lad  - learning rate :  1

Error is on avearage :  0.26763972449390216
Let's say the target value is : 1.0, then the model would guess a value in the following interval :
 -> [ 0.7323602755060978 , 1.2676397244939022 ]


In [13]:
print(properties_txt[1])
print()
print("model -> loss_function : ", best_1[1], " - learning rate : ", best_1[2])
print()
print("Error rate is : ", best_1[0], "%")

Number of rotatable bonds (RBN): the number of bonds which allow free rotation around themselves

model -> max_iter :  ls  - learning rate :  1

Error rate is :  58.43178834696557 %


In [17]:
print(properties_txt[2])
print()
print("model -> loss_function : ", best_2[1], " - learning rate : ", best_2[2])
print()
print("Error is on avearage : ", best_2[0])
print("Let's say the target value is : 150, then the model would guess a value in the following interval :")
print(" -> [", 150.0 - best_2[0],",",150.0 + best_2[0],"]")

Molecular weight (MW): the weight of a molecule based on the atomic masses of all atoms in the molecule

model -> loss_function :  ls  - learning rate :  1

Error is on avearage :  0.6562325422345359
Let's say the target value is : 150, then the model would guess a value in the following interval :
 -> [ 149.34376745776547 , 150.65623254223453 ]


In [16]:
print(properties_txt[3])
print()
print("model -> loss_function : ", best_3[1], " - learning rate : ", best_3[2])
print()
print("Error rate is : ", best_3[0], "%")

Number of the rings (RN): the number of connected sets of atoms and bonds in which every atom and bond is a member of a cycle

model -> loss_function :  huber  - learning rate :  1

Error rate is :  5.399878824598602 %


## comment
Since learning rate has no impact, we changed it for the max_depth parameter

## Tests without learning rate (=1) :

In [22]:
loss_function = ["ls", "lad", "huber", "quantile"]
liste_max_depth = [3, 5, 7, 9]
best_0 = [100,0,0]
best_1 = [100,0,0]
best_2 = [100,0,0]
best_3 = [100,0,0]

#we iterate over loss function and learning rates
for loss in loss_function:
    for max_depth in liste_max_depth:
        # 0 
        reg = GradientBoostingRegressor(random_state=0, learning_rate=1, loss=loss, max_depth=max_depth)
        reg.fit(X_hot,  y_train[:,0])
        y_pred_0 = reg.predict(X_hot_test)
        
        # 1
        reg = GradientBoostingRegressor(random_state=0, learning_rate=1, loss=loss, max_depth=max_depth)
        reg.fit(X_hot,  y_train[:,1])
        y_pred_1 = reg.predict(X_hot_test)
        y_pred_1 = np.round(y_pred_1)

        # 2 
        reg = GradientBoostingRegressor(random_state=0, learning_rate=1, loss=loss, max_depth=max_depth)
        reg.fit(X_hot,  y_train[:,2])
        y_pred_2 = reg.predict(X_hot_test)
        
        # 3 
        reg = GradientBoostingRegressor(random_state=0, learning_rate=1, loss=loss, max_depth=max_depth)
        reg.fit(X_hot,  y_train[:,3])
        y_pred_3 = reg.predict(X_hot_test)
        y_pred_3 = np.round(y_pred_3)
        
        # calcul res
        sum_0 = 0
        sum_1 = 0
        sum_2 = 0
        sum_3 = 0
        for i in range(y_test_0.shape[0]):

            # 0
            sum_0 = sum_0 + np.abs(np.abs(y_pred_0[i]) - np.abs(y_test_0[i]))

            # 1
            if(y_pred_1[i] == y_test_1[i]):
                sum_1 = sum_1 + 1

            # 2
            sum_2 = sum_2 + np.abs(np.abs(y_pred_2[i]) - np.abs(y_test_2[i]))

            # 3
            if(y_pred_3[i] == y_test_3[i]):
                sum_3 = sum_3 + 1

        sum_0 = sum_0 / y_test_0.shape[0]
        sum_1_r = 100 - (sum_1 / y_test_1.shape[0] * 100)
        sum_2 = sum_2 / y_test_2.shape[0]
        sum_3_r = 100 - (sum_3 / y_test_3.shape[0] * 100)
        
        
        # change res if needed
        if sum_0 < best_0[0]:
            best_0 = [sum_0, loss, max_depth]

        if sum_1_r < best_1[0]:
            best_1 = [sum_1_r, loss, max_depth]
            
        if sum_2 < best_2[0]:
            best_2 = [sum_2, loss, max_depth]

        if sum_3_r < best_3[0]:
            best_3 = [sum_3_r, loss, max_depth]

        print(loss, " ", max_depth, "   - done")
        
print()                
print(best_0)
print(best_1)
print(best_2)
print(best_3)

ls   3    - done
ls   5    - done
ls   7    - done
ls   9    - done
lad   3    - done
lad   5    - done
lad   7    - done
lad   9    - done
huber   3    - done
huber   5    - done
huber   7    - done
huber   9    - done
quantile   3    - done
quantile   5    - done
quantile   7    - done
quantile   9    - done

[0.26763972449390216, 'lad', 3]
[58.43178834696557, 'ls', 3]
[0.6153660201806783, 'huber', 7]
[5.399878824598602, 'huber', 3]


## Tests WITH learning rate AND max_depth :
We still want to be sure the learning rate dosen't have an impact when changing the max_depth

In [21]:
loss_function = ["ls", "lad", "huber", "quantile"]
learning_rate_ = [10, 1, 0.1, 0.01, 0.001, 0.0001]
liste_max_depth = [3, 4]
best_0 = [100,0,0,0]
best_1 = [100,0,0,0]
best_2 = [100,0,0,0]
best_3 = [100,0,0,0]

#we iterate over loss function and learning rates
for loss in loss_function:
    for lr in learning_rate_:
        for max_depth in liste_max_depth:

            # 0 
            reg = GradientBoostingRegressor(random_state=0, learning_rate=lr, loss=loss, max_depth=max_depth)
            reg.fit(X_hot,  y_train[:,0])
            y_pred_0 = reg.predict(X_hot_test)

            # 1
            reg = GradientBoostingRegressor(random_state=0, learning_rate=lr, loss=loss, max_depth=max_depth)
            reg.fit(X_hot,  y_train[:,1])
            y_pred_1 = reg.predict(X_hot_test)
            y_pred_1 = np.round(y_pred_1)

            # 2 
            reg = GradientBoostingRegressor(random_state=0, learning_rate=lr, loss=loss, max_depth=max_depth)
            reg.fit(X_hot,  y_train[:,2])
            y_pred_2 = reg.predict(X_hot_test)

            # 3 
            reg = GradientBoostingRegressor(random_state=0, learning_rate=lr, loss=loss, max_depth=max_depth)
            reg.fit(X_hot,  y_train[:,3])
            y_pred_3 = reg.predict(X_hot_test)
            y_pred_3 = np.round(y_pred_3)

            # calcul res
            sum_0 = 0
            sum_1 = 0
            sum_2 = 0
            sum_3 = 0
            for i in range(y_test_0.shape[0]):

                # 0
                sum_0 = sum_0 + np.abs(np.abs(y_pred_0[i]) - np.abs(y_test_0[i]))

                # 1
                if(y_pred_1[i] == y_test_1[i]):
                    sum_1 = sum_1 + 1

                # 2
                sum_2 = sum_2 + np.abs(np.abs(y_pred_2[i]) - np.abs(y_test_2[i]))

                # 3
                if(y_pred_3[i] == y_test_3[i]):
                    sum_3 = sum_3 + 1

            sum_0 = sum_0 / y_test_0.shape[0]
            sum_1_r = 100 - (sum_1 / y_test_1.shape[0] * 100)
            sum_2 = sum_2 / y_test_2.shape[0]
            sum_3_r = 100 - (sum_3 / y_test_3.shape[0] * 100)


            # change res if needed
            if sum_0 < best_0[0]:
                best_0 = [sum_0, loss, lr, max_depth]

            if sum_1_r < best_1[0]:
                best_1 = [sum_1_r, loss, lr, max_depth]

            if sum_2 < best_2[0]:
                best_2 = [sum_2, loss, lr, max_depth]

            if sum_3_r < best_3[0]:
                best_3 = [sum_3_r, loss, lr, max_depth]

            print(loss, " ", lr, " ", max_depth, "   - done")
        
print()                
print(best_0)
print(best_1)
print(best_2)
print(best_3)
        

ls   10   3    - done
ls   10   4    - done
ls   1   3    - done
ls   1   4    - done
ls   0.1   3    - done
ls   0.1   4    - done
ls   0.01   3    - done
ls   0.01   4    - done
ls   0.001   3    - done
ls   0.001   4    - done
ls   0.0001   3    - done
ls   0.0001   4    - done
lad   10   3    - done
lad   10   4    - done
lad   1   3    - done
lad   1   4    - done
lad   0.1   3    - done
lad   0.1   4    - done
lad   0.01   3    - done
lad   0.01   4    - done
lad   0.001   3    - done
lad   0.001   4    - done
lad   0.0001   3    - done
lad   0.0001   4    - done
huber   10   3    - done
huber   10   4    - done
huber   1   3    - done
huber   1   4    - done
huber   0.1   3    - done
huber   0.1   4    - done
huber   0.01   3    - done
huber   0.01   4    - done
huber   0.001   3    - done
huber   0.001   4    - done
huber   0.0001   3    - done
huber   0.0001   4    - done
quantile   10   3    - done
quantile   10   4    - done
quantile   1   3    - done
quantile   1   4    - d

### comment
We see no difference for the learning rate, but a change in the second attribut with max_depth=4

In [23]:
import json #ne jamais faire ça avec les import 

def load_dict_errors(path="all_errors.json"):
    with open(path, "r") as f:
        dict_errors = json.load(f)
        
    return dict_errors

dict_errors_0_2 = load_dict_errors(path="all_errors_0_2.json")
dict_errors_1_3 = load_dict_errors(path="all_errors_1_3.json")
print("dict_errors_0_2", dict_errors_0_2)
print("dict_errors_1_3", dict_errors_1_3)


def save_dict_errors(dict_errors, path="all_errors.json"):
    with open(path, "w") as f:
        json.dump(dict_errors, f)

#dict_errors_0_2['GradBoostRegressor'] = [best_0[0], best_2[0]]
#dict_errors_1_3['GradBoostRegressor'] = [best_1[0], best_3[0]]

dict_errors_0_2['GradBoostRegressor'] = [0.26763972449390216, 0.6153660201806783]
dict_errors_1_3['GradBoostRegressor'] = [58.361102696152685, 5.399878824598602]

print("dict_errors, after changes : ",dict_errors_0_2)
print("dict_errors, after changes : ",dict_errors_1_3)

save_dict_errors(dict_errors_0_2, path="all_errors_0_2.json")
save_dict_errors(dict_errors_1_3, path="all_errors_1_3.json")





dict_errors_0_2 {'lienar_regression': [0.34321702049967723, 0.017421293770233204], 'neural_network': [0.26404827656252755, 2.9858224688390417e-06], 'GradBoostRegressor': [0.26763972449390216, 0.6562325422345359], 'SVR': [0.26732008420568637, 0.8983270988918186]}
dict_errors_1_3 {'GradBoostRegressor': [58.43178834696557, 5.399878824598602], 'SVR': [57.472483085933554, 7.290719983843275], 'lienar_regression': [56.280924972230636, 5.710390790669493], 'neural_network': [45.78915480157528, 35.03483792790064], 'neer_neighboor': [54.907603756437446, 7.081187518933646]}
dict_errors, after changes :  {'lienar_regression': [0.34321702049967723, 0.017421293770233204], 'neural_network': [0.26404827656252755, 2.9858224688390417e-06], 'GradBoostRegressor': [0.26763972449390216, 0.6153660201806783], 'SVR': [0.26732008420568637, 0.8983270988918186]}
dict_errors, after changes :  {'GradBoostRegressor': [58.361102696152685, 5.399878824598602], 'SVR': [57.472483085933554, 7.290719983843275], 'lienar_regr

In [3]:
#clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_hot,  y_train[:,0])
#clf.score(X_hot,  y_train[:,0])
'''
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_hot,  y_train[:,0])
#reg.predict(X_test[1:2])
#reg.score(X_test, y_test)
y_pred_0 = reg.predict(X_hot_test)
print("y_pred_0",y_pred_0)
print("y_pred_0 len", len(y_pred_0))

reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_hot,  y_train[:,1])
y_pred_1 = reg.predict(X_hot_test)

print("y_pred_1",y_pred_1)
print("y_pred_1 len", len(y_pred_1))
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_hot,  y_train[:,2])
y_pred_2 = reg.predict(X_hot_test)

reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_hot,  y_train[:,3])
y_pred_3 = reg.predict(X_hot_test)

'''

y_pred_0 [-0.7190045   1.88391678  1.14099148 ... -0.484946    0.50922493
 -0.40514258]
y_pred_0 len 39612
y_pred_1 [1.32849193 1.024067   1.05879624 ... 0.11713414 0.08986237 0.08591519]
y_pred_1 len 39612


In [4]:
'''y_test_0 = y_test[:,0]
y_test_1 = y_test[:,1]
y_test_2 = y_test[:,2]
y_test_3 = y_test[:,3]   '''

In [5]:
 '''

sum_0 = 0
sum_0_5 = 0
for i in range(y_test_0.shape[0]):
    sum_0 = sum_0 + np.abs(np.abs(y_pred_0[i]) - np.abs(y_test_0[i]))
    if(y_pred_0[i] == y_test_0[i]):
        sum_0_5 = sum_0_5 + 1
sum_0 = sum_0 / y_test_0.shape[0]
sum_0_5_r = sum_0_5 / y_test_0.shape[0] * 100



print("Number of unique values in y_test[:,0] : ", np.unique(y_test_0).shape[0])
print("Min of y_test[:,0] : ", np.min(y_test_0))
print("Max of y_test[:,0] : ", np.max(y_test_0))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_0).shape[0])
print("Min of y_pred : ", np.min(y_pred_0))
print("Max of y_pred : ", np.max(y_pred_0))
print()
print("Error is on avearage : ", sum_0)
print("Let's say the target value is : 1.0, then the model would guess a value in the following interval :")
print(" -> [", 1.0 - sum_0,",",1.0 + sum_0,"]")
print()
print("There were ", sum_0_5, " correct predictions, out of ", y_test_0.shape[0], ", which is ", 100 - sum_0_5_r,"% error rate")
print()

y_pred_0_r = np.around(y_pred_0)

sum_0_r = 0
sum_0_5_r = 0
for i in range(y_pred_0_r.shape[0]):
    sum_0_r = sum_0_r + np.abs(np.abs(y_pred_0_r[i]) - np.abs(y_test_0[i]))
    if(y_pred_0_r[i] == y_test_0[i]):
        sum_0_5_r = sum_0_5_r + 1
sum_0_r = sum_0_r / y_test_0.shape[0]
sum_0_5_r_ = sum_0_5_r / y_test_0.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_0_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_0_r))
print("Max of y_pred_rounded : ", np.max(y_pred_0_r))
print()
print("Error is on avearage : ", sum_0_r)
print("There were ", sum_0_5_r, " correct predictions, out of ", y_test_0.shape[0], ", which is ", 100 - sum_0_5_r_,"% error rate")
'''

Number of unique values in y_test[:,0] :  6400
Min of y_test[:,0] :  -2.8142
Max of y_test[:,0] :  3.7569

Number of unique values in y_pred :  1336
Min of y_pred :  -1.5296909604895872
Max of y_pred :  3.1377478150989435

Error is on avearage :  0.3059346708449163
Let's say the target value is : 1.0, then the model would guess a value in the following interval :
 -> [ 0.6940653291550837 , 1.3059346708449162 ]

There were  0  correct predictions, out of  39612 , which is  100.0 % error rate

Number of unique values in y_pred_rounded :  6
Min of y_pred_rounded :  -2.0
Max of y_pred_rounded :  3.0

Error is on avearage :  0.4064572520574313
There were  2  correct predictions, out of  39612 , which is  99.99495102494194 % error rate


In [6]:
'''sum_1 = 0
sum_1_5 = 0
for i in range(y_test_1.shape[0]):
    sum_1 = sum_1 + np.abs(np.abs(y_pred_1[i]) - np.abs(y_test_1[i]))
    if(y_pred_1[i] == y_test_1[i]):
        sum_1_5 = sum_1_5 + 1
sum_1 = sum_1 / y_test_1.shape[0]
sum_1_5_r = sum_1_5 / y_test_1.shape[0] * 100

print("Number of unique values in y_test[:,1] : ", np.unique(y_test_1).shape[0])
print("Min of y_test[:,1] : ", np.min(y_test_1))
print("Max of y_test[:,1] : ", np.max(y_test_1))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_1).shape[0])
print("Min of y_pred : ", np.min(y_pred_1))
print("Max of y_pred : ", np.max(y_pred_1))
print()
print("Error is on avearage : ", sum_1)
print("There were ", sum_1_5, " correct predictions, out of ", y_test_1.shape[0], ", which is ", 100 - sum_1_5_r,"% error rate")
print()
print("Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : ")
print()

y_pred_1_r = np.around(y_pred_1)

sum_1_r = 0
sum_1_5_r = 0
for i in range(y_pred_1_r.shape[0]):
    sum_1_r = sum_1_r + np.abs(np.abs(y_pred_1_r[i]) - np.abs(y_test_1[i]))
    if(y_pred_1_r[i] == y_test_1[i]):
        sum_1_5_r = sum_1_5_r + 1
sum_1_r = sum_1_r / y_test_1.shape[0]
sum_1_5_r_ = sum_1_5_r / y_test_1.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_1_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_1_r))
print("Max of y_pred_rounded : ", np.max(y_pred_1_r))
print()
print("Error is on avearage : ", sum_1_r)
print("There were ", sum_1_5_r, " correct predictions, out of ", y_test_1.shape[0], ", which is ", 100 - sum_1_5_r_,"% error rate")
'''

Number of unique values in y_test[:,1] :  7
Min of y_test[:,1] :  0.0
Max of y_test[:,1] :  6.0

Number of unique values in y_pred :  1282
Min of y_pred :  -0.48861950331383575
Max of y_pred :  4.1597613162624505

Error is on avearage :  0.7635997201650376
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate

Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : 

Number of unique values in y_pred_rounded :  5
Min of y_pred_rounded :  0.0
Max of y_pred_rounded :  4.0

Error is on avearage :  0.7966777744117944
There were  14237  correct predictions, out of  39612 , which is  64.05887104917701 % error rate


In [7]:
'''sum_2 = 0
sum_2_5 = 0
sum_2_r = 0
for i in range(y_test_2.shape[0]):
    sum_2 = sum_2 + np.abs(np.abs(y_pred_2[i]) - np.abs(y_test_2[i]))
    sum_2_r = sum_2_r + np.abs(100 - (np.abs(y_pred_2[i]) * 100 / np.abs(y_test_2[i])))
    if(y_pred_2[i] == y_test_2[i]):
        sum_2_5 = sum_2_5 + 1
sum_2 = sum_2 / y_test_2.shape[0]
sum_2_r = sum_2_r / y_test_2.shape[0]
sum_2_5_r = sum_2_5 / y_test_2.shape[0] * 100




print("Number of unique values in y_test[:,2] : ", np.unique(y_test_2).shape[0])
print("Min of y_test[:,2] : ", np.min(y_test_2))
print("Max of y_test[:,2] : ", np.max(y_test_2))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_2).shape[0])
print("Min of y_pred : ", np.min(y_pred_2))
print("Max of y_pred : ", np.max(y_pred_2))
print()
print("Error is on avearage : ", sum_2)
print("Error rate is on avearage : ", sum_2_r, "% -> for each guess, the model is of by ", sum_2_r, "%")
print("Let's say the target value is : 150, then the model would guess a value in the following interval :")
print(" -> [", 150 - sum_2,",",150 + sum_2,"]")
print()
print("There were ", sum_2_5, " correct predictions, out of ", y_test_2.shape[0], ", which is ", 100 - sum_2_5_r,"% error rate")
print()

y_pred_2_r = np.around(y_pred_2)

sum_2_r = 0
sum_2_5_r = 0
for i in range(y_pred_2_r.shape[0]):
    sum_2_r = sum_2_r + np.abs(np.abs(y_pred_2_r[i]) - np.abs(y_test_2[i]))
    if(y_pred_2_r[i] == y_test_2[i]):
        sum_2_5_r = sum_2_5_r + 1
sum_2_r = sum_2_r / y_test_2.shape[0]
sum_2_5_r_ = sum_2_5_r / y_test_2.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_2_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_2_r))
print("Max of y_pred_rounded : ", np.max(y_pred_2_r))
print()
print("Error is on avearage : ", sum_2_r)
print("There were ", sum_2_5_r, " correct predictions, out of ", y_test_2.shape[0], ", which is ", 100 - sum_2_5_r_,"% error rate")
'''

Number of unique values in y_test[:,2] :  222
Min of y_test[:,2] :  104.152
Max of y_test[:,2] :  152.037

Number of unique values in y_pred :  1221
Min of y_pred :  107.78899101672297
Max of y_pred :  133.4608887238207

Error is on avearage :  3.882566534691418
Error rate is on avearage :  3.029887968784465 % -> for each guess, the model is of by  3.029887968784465 %
Let's say the target value is : 150, then the model would guess a value in the following interval :
 -> [ 146.11743346530858 , 153.88256653469142 ]

There were  0  correct predictions, out of  39612 , which is  100.0 % error rate

Number of unique values in y_pred_rounded :  25
Min of y_pred_rounded :  108.0
Max of y_pred_rounded :  133.0

Error is on avearage :  3.907632413854698
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate


In [8]:
'''sum_3 = 0
sum_3_5 = 0
for i in range(y_test_3.shape[0]):
    sum_3 = sum_3 + np.abs(np.abs(y_pred_3[i]) - np.abs(y_test_3[i]))
    if(y_pred_3[i] == y_test_3[i]):
        sum_3_5 = sum_3_5 + 1
sum_3 = sum_3 / y_test_1.shape[0]
sum_3_5_r = sum_3_5 / y_test_1.shape[0] * 100

print("Number of unique values in y_test[:,3] : ", np.unique(y_test_3).shape[0])
print("Min of y_test[:,3] : ", np.min(y_test_3))
print("Max of y_test[:,3] : ", np.max(y_test_3))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_3).shape[0])
print("Min of y_pred : ", np.min(y_pred_3))
print("Max of y_pred : ", np.max(y_pred_3))
print()
print("Error is on avearage : ", sum_3)
print("There were ", sum_3_5, " correct predictions, out of ", y_test_3.shape[0], ", which is ", 100 - sum_3_5_r,"% error rate")
print()
print("Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : ")
print()

y_pred_3_r = np.around(y_pred_3)

sum_3_r = 0
sum_3_5_r = 0
for i in range(y_test_3.shape[0]):
    sum_3_r = sum_3_r + np.abs(np.abs(y_pred_3_r[i]) - np.abs(y_test_3[i]))
    if(y_pred_3_r[i] == y_test_3[i]):
        sum_3_5_r = sum_3_5_r + 1
sum_3_r = sum_3_r / y_test_3.shape[0]
sum_3_5_r_ = sum_3_5_r / y_test_3.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_3_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_3_r))
print("Max of y_pred_rounded : ", np.max(y_pred_3_r))
print()
print("Error is on avearage : ", sum_3_r)
print("There were ", sum_3_5_r, " correct predictions, out of ", y_test_3.shape[0], ", which is ", 100 - sum_3_5_r_,"% error rate")
'''

Number of unique values in y_test[:,3] :  9
Min of y_test[:,3] :  0.0
Max of y_test[:,3] :  8.0

Number of unique values in y_pred :  650
Min of y_pred :  -0.001379710303792963
Max of y_pred :  6.610218580449615

Error is on avearage :  0.1045783467369218
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate

Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : 

Number of unique values in y_pred_rounded :  8
Min of y_pred_rounded :  0.0
Max of y_pred_rounded :  7.0

Error is on avearage :  0.0725790164596587
There were  36887  correct predictions, out of  39612 , which is  6.879228516611121 % error rate


In [9]:
'''import json #ne jamais faire ça avec les import 

def load_dict_errors(path="all_errors.json"):
    with open(path, "r") as f:
        dict_errors = json.load(f)
        
    return dict_errors

dict_errors = load_dict_errors()'''

In [10]:
'''def save_dict_errors(dict_errors, path="all_errors.json"):
    with open(path, "w") as f:
        json.dump(dict_errors, f)

        '''

In [11]:
'''
#dict_errors['GradBoostRegressor'] = [100 - sum_0_5_r_, 100 - sum_1_5_r_, 100 - sum_2_5_r_, 100 - sum_3_5_r_]
dict_errors['GradBoostRegressor'] = [0, 100 - sum_1_5_r_, 0, 100 - sum_3_5_r_]

#dict_errors = { "GradBoostRegressor" : [100 - sum_0_5_r_, 100 - sum_1_5_r_, 100 - sum_2_5_r_, 100 - sum_3_5_r_]}

print(dict_errors)
save_dict_errors(dict_errors)'''

{'GradBoostRegressor': [0, 64.05887104917701, 0, 6.879228516611121], 'LinearRegression': [0, 57.64162375037867, 0, 7.911743915985056], 'SVR': [0, 57.472483085933554, 0, 7.290719983843275]}


dict_errors_0_2 {'lienar_regression': [0.34321702049967723, 0.017421293770233204], 'neural_network': [0.26404827656252755, 2.9858224688390417e-06], 'SGDRegressor': [0.35036686833289443, 0.010050606676439778]}
dict_errors_1_3 {'GradBoostRegressor': [64.05887104917701, 6.879228516611121], 'SVR': [57.472483085933554, 7.290719983843275], 'lienar_regression': [56.280924972230636, 5.710390790669493], 'neural_network': [45.78915480157528, 35.03483792790064], 'neer_neighboor': [54.907603756437446, 7.081187518933646], 'SGDRegressor': [56.52327577501767, 7.01050186812077]}
dict_errors, after changes :  {'lienar_regression': [0.34321702049967723, 0.017421293770233204], 'neural_network': [0.26404827656252755, 2.9858224688390417e-06], 'SGDRegressor': [0.35036686833289443, 0.010050606676439778], 'GradBoostRegressor': [0.3059346708449163, 3.882566534691418]}
dict_errors, after changes :  {'GradBoostRegressor': [64.05887104917701, 6.879228516611121], 'SVR': [57.472483085933554, 7.290719983843275], 'li